In [10]:
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import os

import folium

In [11]:
# Connect to the dev.duckdb database located in the dbt_pollution/ directory
db_path = os.path.join(os.path.dirname(os.getcwd()), 'dev.duckdb')
conn = duckdb.connect(db_path)
print(f'Connected to database at {db_path} successfully!')

# Show available tables
tables = conn.execute('SHOW TABLES').fetchall()
print('\nAvailable tables:')
for table in tables:
    print(f'- {table[0]}')

Connected to database at /Users/MPC/Documents/Github-Personal/pollution-prediction/dbt_pollution/dev.duckdb successfully!

Available tables:
- instrument_data
- lnd_instrument_data
- lnd_measurements
- lnd_pollutants
- measurement_data
- measurements_with_status
- pollutant_data


In [12]:
# Load and examine the measurements_with_status table
print("=== MEASUREMENTS WITH STATUS - PRIMARY ANALYSIS TABLE ===")
print("Loading the consolidated table with measurements and instrument status...")

df = conn.execute("SELECT * FROM measurements_with_status").df()

print(f"✅ Dataset loaded successfully!")
print(f"   📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"   📅 Date range: {df['measurement_datetime'].min()} to {df['measurement_datetime'].max()}")
print(f"   🏭 Stations: {df['station_code'].nunique()} unique stations")
print(f"   💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print(f"\n📋 Column Information:")
print(f"   - measurement_datetime: {df['measurement_datetime'].dtype}")
print(f"   - station_code: {df['station_code'].dtype} (range: {df['station_code'].min()}-{df['station_code'].max()})")
print(f"   - coordinates: latitude, longitude (float64)")
print(f"   - pollutants: so2_value, no2_value, o3_value, co_value, pm10_value, pm2_5_value")
print(f"   - quality: instrument_status ({df['instrument_status'].dtype})")

display(df.head())

=== MEASUREMENTS WITH STATUS - PRIMARY ANALYSIS TABLE ===
Loading the consolidated table with measurements and instrument status...
✅ Dataset loaded successfully!
   📊 Shape: 3,707,973 rows × 11 columns
   📅 Date range: 2021-01-01 00:00:00 to 2023-12-31 23:00:00
   🏭 Stations: 25 unique stations
   💾 Memory usage: 286.4 MB

📋 Column Information:
   - measurement_datetime: datetime64[us]
   - station_code: int32 (range: 204-228)
   - coordinates: latitude, longitude (float64)
   - pollutants: so2_value, no2_value, o3_value, co_value, pm10_value, pm2_5_value
   - quality: instrument_status (Int32)


,measurement_datetime,station_code,latitude,longitude,so2_value,no2_value,o3_value,co_value,pm10_value,pm2_5_value,instrument_status
0,2021-01-01,204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,0
1,2021-01-01,204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,0
2,2021-01-01,204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,0
3,2021-01-01,204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,0
4,2021-01-01,204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,0


In [13]:
# Data Quality Assessment - Critical Issues Analysis
print("=== DATA QUALITY ASSESSMENT ===")
print("Analyzing the critical data quality issues identified...")

# 1. Missing Values (-1) Analysis
pollutant_cols = ['so2_value', 'no2_value', 'o3_value', 'co_value', 'pm10_value', 'pm2_5_value']

print("🔍 1. MISSING VALUES ANALYSIS (-1 represents null)")
print("-" * 50)
missing_analysis = {}
for col in pollutant_cols:
    missing_count = (df[col] == -1).sum()
    missing_pct = (missing_count / len(df)) * 100
    missing_analysis[col] = {'count': missing_count, 'percentage': missing_pct}
    print(f"   {col.upper()}: {missing_count:,} missing ({missing_pct:.2f}%)")

total_missing = sum([v['count'] for v in missing_analysis.values()])
print(f"   📊 TOTAL MISSING VALUES: {total_missing:,} ({(total_missing/(len(df)*6))*100:.2f}% of all measurements)")

# 2. Missing Instrument Status Analysis  
print(f"\n🔍 2. MISSING INSTRUMENT STATUS ANALYSIS")
print("-" * 50)
null_status_count = df['instrument_status'].isna().sum()
null_status_pct = (null_status_count / len(df)) * 100
print(f"   Missing instrument_status: {null_status_count:,} records ({null_status_pct:.2f}%)")
print(f"   Records with status: {len(df) - null_status_count:,} records ({100-null_status_pct:.2f}%)")

# 3. Instrument Status Distribution (for non-null values)
print(f"\n🔍 3. INSTRUMENT STATUS DISTRIBUTION")
print("-" * 50)
status_counts = df['instrument_status'].value_counts(dropna=False).sort_index()
status_labels = {
    0: "Normal",
    1: "Need calibration", 
    2: "Abnormal",
    4: "Power cut off",
    8: "Under repair",
    9: "Abnormal data"
}

print("   Status distribution:")
for status, count in status_counts.items():
    if pd.isna(status):
        print(f"   NULL: {count:,} ({count/len(df)*100:.2f}%)")
    else:
        label = status_labels.get(status, f"Unknown ({status})")
        print(f"   {status} ({label}): {count:,} ({count/len(df)*100:.2f}%)")

# 4. Complete Records Analysis
print(f"\n🔍 4. COMPLETE RECORDS ANALYSIS")
print("-" * 50)

# Records with no missing values and valid instrument status
no_missing_values = df[~df[pollutant_cols].isin([-1]).any(axis=1)]
complete_records = no_missing_values[no_missing_values['instrument_status'].notna()]
high_quality_records = complete_records[complete_records['instrument_status'] == 0]

print(f"   📊 Records without -1 values: {len(no_missing_values):,} ({len(no_missing_values)/len(df)*100:.1f}%)")
print(f"   📊 Complete records (values + status): {len(complete_records):,} ({len(complete_records)/len(df)*100:.1f}%)")
print(f"   ✅ High quality records (complete + normal status): {len(high_quality_records):,} ({len(high_quality_records)/len(df)*100:.1f}%)")

=== DATA QUALITY ASSESSMENT ===
Analyzing the critical data quality issues identified...
🔍 1. MISSING VALUES ANALYSIS (-1 represents null)
--------------------------------------------------
   SO2_VALUE: 19,523 missing (0.53%)
   NO2_VALUE: 18,671 missing (0.50%)
   O3_VALUE: 19,985 missing (0.54%)
   CO_VALUE: 19,847 missing (0.54%)
   PM10_VALUE: 19,189 missing (0.52%)
   PM2_5_VALUE: 19,309 missing (0.52%)
   📊 TOTAL MISSING VALUES: 116,524 (0.52% of all measurements)

🔍 2. MISSING INSTRUMENT STATUS ANALYSIS
--------------------------------------------------
   Missing instrument_status: 4,311 records (0.12%)
   Records with status: 3,703,662 records (99.88%)

🔍 3. INSTRUMENT STATUS DISTRIBUTION
--------------------------------------------------
   Status distribution:
   0 (Normal): 3,606,696 (97.27%)
   1 (Need calibration): 27,147 (0.73%)
   2 (Abnormal): 3,868 (0.10%)
   4 (Power cut off): 17,960 (0.48%)
   8 (Under repair): 28,323 (0.76%)
   9 (Abnormal data): 19,668 (0.53%)
  

In [16]:
# Get unique stations with their coordinates
station_locations = (
    df[['station_code', 'latitude', 'longitude']]
    .drop_duplicates(subset=['station_code'])
    .reset_index(drop=True)
)

# Calculate the center of the map
center_lat = station_locations['latitude'].mean()
center_lon = station_locations['longitude'].mean()

# Create a folium map centered at the average location
m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# Add a marker for each station
for _, row in station_locations.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Station {row['station_code']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# Display the map
display(m)

**Station Location Information**

- All 25 monitoring stations in this dataset are located in **South Korea**, most likely within the Seoul metropolitan area.
- The stations are distributed across various districts, providing comprehensive air quality coverage for the region.
- Each station is identified by a unique `station_code` and has associated geographic coordinates (`latitude`, `longitude`).

**Other Details:**
- The dataset covers years between 2021 and 2023 and includes hourly measurements for key pollutants (SO₂, NO₂, O₃, CO, PM10, PM2.5).
- The spatial distribution of stations allows for detailed analysis of air quality patterns across urban and suburban areas.
- The latitude and longitude values suggest all stations are within a relatively compact urban area, supporting high-resolution spatial analysis.

In [ ]:
# Save clean dataset for Streamlit application
print("=== PREPARING DATA FOR STREAMLIT APPLICATION ===")

# Create final clean dataset
final_clean_df = df[
    (df['instrument_status'].notna()) &  # Has instrument status
    (~df[pollutant_cols].isin([-1]).any(axis=1))  # No missing values (-1)
].copy()

# Add useful features for Streamlit
final_clean_df['year'] = final_clean_df['measurement_datetime'].dt.year
final_clean_df['month'] = final_clean_df['measurement_datetime'].dt.month
final_clean_df['day'] = final_clean_df['measurement_datetime'].dt.day
final_clean_df['hour'] = final_clean_df['measurement_datetime'].dt.hour
final_clean_df['day_of_week'] = final_clean_df['measurement_datetime'].dt.day_of_week
final_clean_df['season'] = final_clean_df['month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring', 
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
})

# Quality categories
final_clean_df['data_quality'] = final_clean_df['instrument_status'].map({
    0: 'Normal',
    1: 'Calibration Needed',
    2: 'Abnormal',
    4: 'Power Cut',
    8: 'Under Repair', 
    9: 'Bad Data'
})

print(f"✅ Clean dataset prepared:")
print(f"   📊 Records: {len(final_clean_df):,} ({len(final_clean_df)/len(df)*100:.1f}% of original)")
print(f"   📅 Date range: {final_clean_df['measurement_datetime'].min()} to {final_clean_df['measurement_datetime'].max()}")
print(f"   🏭 Stations: {final_clean_df['station_code'].nunique()}")
print(f"   ✅ Normal status records: {(final_clean_df['instrument_status'] == 0).sum():,}")

# Show columns available for Streamlit
print(f"\n📋 Available columns for Streamlit app:")
print(f"   🕒 Temporal: year, month, day, hour, day_of_week, season")
print(f"   📍 Spatial: station_code, latitude, longitude") 
print(f"   💨 Pollutants: {', '.join([col.replace('_value', '').upper() for col in pollutant_cols])}")
print(f"   🔍 Quality: instrument_status, data_quality")

# Sample for verification
print(f"\nSample of clean data:")
display(final_clean_df[['measurement_datetime', 'station_code', 'so2_value', 'no2_value', 'pm10_value', 'season', 'data_quality']].head())

print(f"\n🚀 Ready for Streamlit development!")
print(f"   Use this clean dataset: final_clean_df")
print(f"   Recommended filters: station_code, date range, data_quality, season")
print(f"   Key visualizations: time series, geographic maps, correlation analysis")

=== PREPARING DATA FOR STREAMLIT APPLICATION ===
✅ Clean dataset prepared:
   📊 Records: 3,678,684 (99.2% of original)
   📅 Date range: 2021-01-01 00:00:00 to 2023-12-31 23:00:00
   🏭 Stations: 25
   ✅ Normal status records: 3,602,080

📋 Available columns for Streamlit app:
   🕒 Temporal: year, month, day, hour, day_of_week, season
   📍 Spatial: station_code, latitude, longitude
   💨 Pollutants: SO2, NO2, O3, CO, PM10, PM2_5
   🔍 Quality: instrument_status, data_quality

Sample of clean data:


,measurement_datetime,station_code,so2_value,no2_value,pm10_value,season,data_quality
0,2021-01-01,204,0.004,0.059,73.0,Winter,Normal
1,2021-01-01,204,0.004,0.059,73.0,Winter,Normal
2,2021-01-01,204,0.004,0.059,73.0,Winter,Normal
3,2021-01-01,204,0.004,0.059,73.0,Winter,Normal
4,2021-01-01,204,0.004,0.059,73.0,Winter,Normal



🚀 Ready for Streamlit development!
   Use this clean dataset: final_clean_df
   Recommended filters: station_code, date range, data_quality, season
   Key visualizations: time series, geographic maps, correlation analysis


: 